In [9]:
# _*_ coding=utf-8 _*_
import pre  # 方法文件
import json  # 读取json数据集
import jieba  # 分词
from torch.utils.data import DataLoader # 分批处理器
from transformers import BertTokenizer, BertModel   # 导入模型
from torch.nn.functional import cosine_similarity
from scipy.spatial.distance import euclidean    # 欧氏距离
import re   # 正则表达式
import pandas as pd # csv读取处理
import numpy as np
import torch
import csv

In [10]:
# 读取train.json文件
with open(
    "/home/nekozo/VSCode/#Programs/Python/Task/Diag_Nor/dataset/train.json",
    "r",
    encoding="utf-8",
) as file:
    json_data = json.load(file)
# 将预训练数据导入字典
result_dict = {}
for item in json_data:
    result_dict[item["text"]] = item["normalized_result"]

# 创建列表存储未处理的数据
nonP_text = list(result_dict.keys())
nonP_result = list(result_dict.values())
nonP_result = [item.split("##") for item in nonP_result]    # 以“##”分割
str_result = []
for item in nonP_result:    # 去重
    for sentence in item:
        if sentence not in str_result:
            str_result.append(sentence)

# 数据规范化(去除非中文字符)
s_text = [pre.delete_punctuations(item) for item in str_result]
st_text = []
for s in s_text:
    str_s = ''.join(s)
    st_text.append(str_s)
    
# 获取停止词列表
with open(
    "/home/nekozo/VSCode/#Programs/Python/Task/Diag_Nor/dataset/stop_word.txt", "r"
) as f:
    st_words = f.readlines()
st_words = [item.strip() for item in st_words]  # 以\n分割

# 去除空值与停止词
sp_text = [item for item in st_text if item]  # remove NULL
final_train = []
for sentence in sp_text:
    p_sentence = list(jieba.cut(sentence))
    p_sentence = [
        item for item in p_sentence if item not in st_words
    ]  # remove word in st_words
    text = "".join(p_sentence)  # transfrom to string
    final_train.append(text)

In [11]:
# 加载模型
model_path='/home/nekozo/VSCode/model/bert-base-chinese/'
tokenizer = BertTokenizer.from_pretrained(model_path, num_threads=8)
model = BertModel.from_pretrained(model_path)

In [12]:
# 计算normalized_result词嵌入
i = 0
lens = len(final_train)
embedded_train = []
print("Calculating Start!")
for item in final_train:
    i += 1 
    print(str(i)+'/'+str(lens))
    item_list = [item]
    embedded_item = []
    train_dataset = pre.TextDataset(item_list, tokenizer, max_length=128)
    train_Dataloader = DataLoader(train_dataset, batch_size=4, shuffle=False, num_workers=4, collate_fn=pre.collate_fn) 
    for batch in train_Dataloader:
        embedded_text = pre.bert_embedding(batch, model)
        embedded_item.append(embedded_text)
    embedded_train.append(torch.cat(embedded_item, dim=0))

Calculating Start!
1/3503
2/3503
3/3503
4/3503
5/3503
6/3503
7/3503
8/3503
9/3503
10/3503
11/3503
12/3503
13/3503
14/3503
15/3503
16/3503
17/3503
18/3503
19/3503
20/3503
21/3503
22/3503
23/3503
24/3503
25/3503
26/3503
27/3503
28/3503
29/3503
30/3503
31/3503
32/3503
33/3503
34/3503
35/3503
36/3503
37/3503
38/3503
39/3503
40/3503
41/3503
42/3503
43/3503
44/3503
45/3503
46/3503
47/3503
48/3503
49/3503
50/3503
51/3503
52/3503
53/3503
54/3503
55/3503
56/3503
57/3503
58/3503
59/3503
60/3503
61/3503
62/3503
63/3503
64/3503
65/3503
66/3503
67/3503
68/3503
69/3503
70/3503
71/3503
72/3503
73/3503
74/3503
75/3503
76/3503
77/3503
78/3503
79/3503
80/3503
81/3503
82/3503
83/3503
84/3503
85/3503
86/3503
87/3503
88/3503
89/3503
90/3503
91/3503
92/3503
93/3503
94/3503
95/3503
96/3503
97/3503
98/3503
99/3503
100/3503
101/3503
102/3503
103/3503
104/3503
105/3503
106/3503
107/3503
108/3503
109/3503
110/3503
111/3503
112/3503
113/3503
114/3503
115/3503
116/3503
117/3503
118/3503
119/3503
120/3503
121/3503


In [13]:
# 将embedded_train写入csv
i = 0
print("Writting Start!")
fieldname = ["ID", "DATA"]
with open("filtered_train_data.csv", mode="a", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=fieldname)
    writer.writeheader()
for text in embedded_train:
    i += 1
    print(str(i) + "/" + str(lens))
    with open("filtered_train_data.csv", mode="a", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=fieldname)
        writer.writerow({"ID": i, "DATA": text})

Writting Start!
1/3503
2/3503
3/3503
4/3503
5/3503
6/3503
7/3503
8/3503
9/3503
10/3503
11/3503
12/3503
13/3503
14/3503
15/3503
16/3503
17/3503
18/3503
19/3503
20/3503
21/3503
22/3503
23/3503
24/3503
25/3503
26/3503
27/3503
28/3503
29/3503
30/3503
31/3503
32/3503
33/3503
34/3503
35/3503
36/3503
37/3503
38/3503
39/3503
40/3503
41/3503
42/3503
43/3503
44/3503
45/3503
46/3503
47/3503
48/3503
49/3503
50/3503
51/3503
52/3503
53/3503
54/3503
55/3503
56/3503
57/3503
58/3503
59/3503
60/3503
61/3503
62/3503
63/3503
64/3503
65/3503
66/3503
67/3503
68/3503
69/3503
70/3503
71/3503
72/3503
73/3503
74/3503
75/3503
76/3503
77/3503
78/3503
79/3503
80/3503
81/3503
82/3503
83/3503
84/3503
85/3503
86/3503
87/3503
88/3503
89/3503
90/3503
91/3503
92/3503
93/3503
94/3503
95/3503
96/3503
97/3503
98/3503
99/3503
100/3503
101/3503
102/3503
103/3503
104/3503
105/3503
106/3503
107/3503
108/3503
109/3503
110/3503
111/3503
112/3503
113/3503
114/3503
115/3503
116/3503
117/3503
118/3503
119/3503
120/3503
121/3503
122

In [14]:
# 读取CSV文件
df = pd.read_csv("/home/nekozo/VSCode/#Programs/Python/Task/Diag_Nor/dataset/filtered_train_data.csv")
# 初始化字典
embedded_train_dict = {}
# 定义正则表达式来提取DATA中的数值部分
for index, row in df.iterrows():
    string = row["DATA"]
    data_list = [float(num) for num in re.findall(r'[-+]?\d*(?:\.\d+)?(?:[eE][-+]?\d+)?', string) if num]
    embedded_train_dict[row["ID"]] = np.array(data_list)

In [15]:
# 获取test文件内容
with open('/home/nekozo/VSCode/#Programs/Python/Task/Diag_Nor/dataset/dev.json', 'r', encoding='utf-8') as file:
    test_data = json.load(file)
test_dict = {}
for item in test_data:
    test_dict[item["text"]] = item["normalized_result"]
test_text_list = list(test_dict.keys())
print(f"test_text_list的长度为{len(test_text_list)}")

# 对test中的字符串进行预处理，准备计算词嵌入
s_test = [pre.delete_punctuations(item) for item in test_text_list] # 去除非中文
st_test = []
for s in s_test:
    str_s = ''.join(s)
    st_test.append(str_s)
    
final_test_text = []
for sentence in st_test:    # 去除停止词
    p_sentence = list(jieba.cut(sentence))
    p_sentence = [
        item for item in p_sentence if item not in st_words
    ]  # remove word in st_words
    text = "".join(p_sentence)  # transfrom to string
    final_test_text.append(text)
print(f"final_test_text的长度为{len(final_test_text)}")
print(final_test_text)

test_text_list的长度为2000
final_test_text的长度为2000
['型糖尿病性植物神经病变', '卵巢Ca', '髂腰肌囊性占位', '子宫颈鳞癌IIB期放化疗', '肿标升高待查全麻胃肠镜丙泊酚耐受', '无精症输精管梗阻精液淤积症', '乳癌术后定期化疗门诊开药', '右膝关节骨关节炎', '急性B淋巴细胞白血病L2型标危 CR化疗', '高甘油三酯脂血症前列腺梗阻未特指', '卵巢癌肿瘤细胞减灭术术后', '右膝外翻重度左膝翻轻度', '双叶甲状腺癌', '胸椎破坏伴截瘫', '十二指肠降段隆起性病变', '急慢性炎伴淋巴组织增生', '糖尿病糖尿病周围神经病变糖尿病肾病IV期', '惊厥', '卵圆孔未闭发泡实验阳性', '左肾缺失', '门诊胃窦管状腺瘤收入院', '左腋窝肉芽组织肿', '完全性肺静脉异位引流心下型梗阻', '右输尿管结石右肾积液', '胸中段食管鳞癌化疗', '肺曲霉病', '右肺叶型腺癌', '晚孕上感', '下颌骨发育不全', '左足背砸伤软组织损伤', '骨转移左11肋右肋', '新生儿先天性心血管畸形', '母体妊娠糖尿病新生儿', '语言迟缓铁低锌低', '急性胃肠道功能不全', '全身脏器功能衰竭心肝肾肺胰腺凝血肠道', '右骨盆肿物', '右髌骨软化右股骨外髁上方局灶性囊性变', '非胰岛素依赖型糖尿病伴有酮症酸中毒', '结肠溃疡型分化腺癌', '宫颈细胞癌IIA2期', '骶尾骨中部骨折', '头痛口干肝肾', '进行性掌指角皮病', '先天性心脏病合并妊娠', '右侧肋间血管破裂', '颅面骨发育不全', '段胸骨伤口愈合不良', '低级别滤泡性淋巴瘤IV期B组', '急性脑梗死右侧颞枕叶出血性梗死', '右前臂带状孢疹', '小房缺染色体病', '双侧髂动脉右侧股浅动脉支架植入术', '腰背部劳损', '双髋关节退行性改变', '骨关节病坐骨神经痛肌肉关节酸痛行动不便', '诺卡菌病肺皮肤骨', '右眼假性剥脱综合症', '肛管末端代谢局灶性增高痔疮可能性', '甲状腺右叶乳头状微癌', '左手中指恶性黑素瘤', '右足跟部皮肤软组织脱套伤', '缺锌重轻度缺铁胃口睡眠差', '精神异常原因待查', '胸片提示右上肺可疑结节', '肺特指疾患双肺叶斑片影', '双下肢行走异常', '右

In [16]:
# 计算test_text的词嵌入,做相似度匹配，将输出存入列表
i = 0
lens = len(final_test_text)
embedded_test_dict = {}
most_similar_list = []
print("Calculating Start!")
for item in final_test_text:
    i += 1 
    print(str(i)+'/'+str(lens))
    item_list = [item]
    embedded_item = []
    test_dataset = pre.TextDataset(item_list, tokenizer, max_length=128)
    test_Dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=4, collate_fn=pre.collate_fn) 
    for batch in test_Dataloader:
        embedded_text = pre.bert_embedding(batch, model)
    embedded_test = str(embedded_text)
    # 将数值提取出来，转换为numpy_array
    data_list = [float(num) for num in re.findall(r'[-+]?\d*(?:\.\d+)?(?:[eE][-+]?\d+)?', embedded_test) if num]
    test_array = np.array(data_list)
    # 计算相似度，返回相似度最高的ID
    similarities = []
    for key, value in embedded_train_dict.items():
        text1 = torch.tensor(test_array).view(1, -1)   # 将词嵌入数据转化为二维张量
        text2 = torch.tensor(value).view(1, -1)
        similarity =cosine_similarity(text1, text2).item() # 计算余弦相似度
        similarities.append(similarity)
    # 找到相似度最高的索引
    max_similarity_index = similarities.index(max(similarities))
    # 获取相应的nonP_result项
    most_similar_result = str_result[max_similarity_index]
    print(item)
    print(f"{most_similar_result} : {max(similarities)}")
    most_similar_list.append(most_similar_result)

Calculating Start!
1/2000
型糖尿病性植物神经病变
高同型半胱氨酸血症 : 0.9485692673799953
2/2000
卵巢Ca
耳其他特指的疾患 : 0.8122785381336133
3/2000
髂腰肌囊性占位
先天性双侧隐睾 : 0.8667793838313421
4/2000
子宫颈鳞癌IIB期放化疗
髋关节囊肿 : 0.865273398947561
5/2000
肿标升高待查全麻胃肠镜丙泊酚耐受
胫动脉闭塞 : 0.821668136970282
6/2000
无精症输精管梗阻精液淤积症
癔症样发作 : 0.8673295555353623
7/2000
乳癌术后定期化疗门诊开药
粘液癌 : 0.8791734763147449
8/2000
右膝关节骨关节炎
腭恶性肿瘤 : 0.9521913357710312
9/2000
急性B淋巴细胞白血病L2型标危 CR化疗
急性淋巴细胞白血病L2 : 0.9117039466144955
10/2000
高甘油三酯脂血症前列腺梗阻未特指
胸骨骨折 : 0.8813430651648861
11/2000
卵巢癌肿瘤细胞减灭术术后
食管恶性肿瘤 : 0.8926062963608151
12/2000
右膝外翻重度左膝翻轻度
抽动秽语综合征 : 0.827422933589459
13/2000
双叶甲状腺癌
冠状动脉搭桥术后 : 0.920383646407607
14/2000
胸椎破坏伴截瘫
骨盆部损伤 : 0.8749895853024943
15/2000
十二指肠降段隆起性病变
上肢瘢痕 : 0.8895782286221299
16/2000
急慢性炎伴淋巴组织增生
B细胞急性淋巴细胞白血病 : 0.9221705736374612
17/2000
糖尿病糖尿病周围神经病变糖尿病肾病IV期
颈椎管狭窄 : 0.9045675504450866
18/2000
惊厥
后天性耳廓缺损 : 0.8960226900746004
19/2000
卵圆孔未闭发泡实验阳性
卵圆孔未闭 : 0.8025468913767801
20/2000
左肾缺失
胃底静脉曲张 : 0.8615106729239976
21/2000
门诊胃窦管状腺瘤收入院
晕厥 : 0.842474

In [17]:
# 将test_text_list与most_similar_list链接成字典
print(f"test_text_list的长度为{len(test_text_list)}")
print(f"most_similar_list的长度为{len(most_similar_list)}")

for j in range(5):
    print(test_text_list[j])
    print(most_similar_list[j])
    print(" ")
list_of_dicts = [{"text": key, "normalized_result": value} for key, value in zip(test_text_list, most_similar_list)]
data = json.dumps(list_of_dicts, ensure_ascii=False)
with open('tested_dev.json', 'w', encoding='utf-8') as json_file:
    json_file.write('[\n')

    # 将数据按行写入JSON文件，每个字典之间加逗号
    for i, item in enumerate(list_of_dicts):
        json_file.write(json.dumps(item, ensure_ascii=False))
        if i < len(list_of_dicts) - 1:
            json_file.write(',\n')

    # 在写入文件之后加上右方括号
    json_file.write('\n]')
print("测试结果文件tested_dev.json已生成")


test_text_list的长度为2000
most_similar_list的长度为2000
1型糖尿病性植物神经病变
高同型半胱氨酸血症
 
卵巢Ca
耳其他特指的疾患
 
髂腰肌囊性占位
先天性双侧隐睾
 
子宫颈鳞癌IIB期放化疗后
髋关节囊肿
 
肿标升高待查全麻胃肠镜丙泊酚不耐受
胫动脉闭塞
 
测试结果文件tested.json已生成
